<a href="https://colab.research.google.com/github/Whoami666/HSE_Lab/blob/main/clean_models/SMOTE_MLPClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 KB 3.6 MB/s eta 0:00:00


In [3]:
%cd drive
%cd MyDrive
%cd shiz

/content/drive
/content/drive/MyDrive
/content/drive/MyDrive/shiz


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

In [5]:
df = pd.read_excel('df_standard_cleaned_21feb.xlsx') 
df1d = pd.read_excel('df_standard_only1dfeatures.xlsx')

df = df.set_index('ID')
df1d = df1d.set_index('ID')
df.head(1)

,sex,tds,chroma_cens_avg,chroma_cens_std,chroma_cqt_avg,chroma_cqt_std,chroma_stft_avg,chroma_stft_std,melspectrogram_avg,melspectrogram_std,...,spectral_flatness_std,spectral_rolloff_avg,spectral_rolloff_std,tempogram_avg,tempogram_std,tonnetz_avg,tonnetz_std,zero_crossing_rate_avg,zero_crossing_rate_std,tds_binary
ID,,,,,,,,,,,,,,,,,,,,,
PD-003,1,0,0.685706,0.428088,0.425039,0.296394,0.339246,0.434412,0.327867,0.348849,...,0.31499,0.397736,0.254215,0.747885,0.719312,0.691479,0.329593,0.197304,0.281692,0


Models

In [10]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.inspection import permutation_importance

from sklearn.linear_model import LogisticRegression
from skopt import BayesSearchCV
from sklearn.svm import SVC

from sklearn.datasets import load_iris
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_digits

from sklearn.model_selection import GridSearchCV

In [11]:
from imblearn.over_sampling import SMOTE

In [7]:
def calculate_scores(model, givenx, giveny, xtest):
  kf = KFold(n_splits=10)
  crossval_f1 = cross_val_score(model, givenx, giveny, scoring='f1_macro', cv = kf, verbose=False)
  mean_score = crossval_f1.mean()
  std_score = crossval_f1.std()
  print('f1_macro', mean_score, '±', std_score)

  crossval_accuracy = cross_val_score(model, givenx, giveny, scoring='accuracy', cv = kf, verbose=False)
  mean_score = crossval_accuracy.mean()
  std_score = crossval_accuracy.std()
  print('accuracy', mean_score, '±', std_score)

  crossval_recall = cross_val_score(model, givenx, giveny, scoring='recall_macro', cv = kf, verbose=False)
  mean_score = crossval_recall.mean()
  std_score = crossval_recall.std()
  print('recall', mean_score, '±', std_score)

  if giveny.nunique() == 2:
    y_proba = model.predict_proba(xtest)
    print('roc_auc_score', roc_auc_score(y_test, y_proba[:,1]))

In [8]:
def mlp_class(xtrain, ytrain, xtest, ytest, xgiven, ygiven):
  sc_X = StandardScaler()
  X_trainscaled=sc_X.fit_transform(xtrain)
  X_testscaled=sc_X.transform(xtest)

  warnings.filterwarnings('ignore')
  opt = BayesSearchCV(
      MLPClassifier(),
      {
          'activation': ['identity', 'logistic', 'tanh', 'relu'],
          'random_state': [1, 2, 3],
          'solver': ['sgd', 'adam']
      },
      n_iter=32,
      scoring='f1_macro',
      cv=3
  )

  opt.fit(X_trainscaled, y_train)

  print("val. score: %s" % opt.best_score_)
  print("best_params: %s" % opt.best_params_, '\n')
  print("test score: %s" % opt.score(X_testscaled, y_test))

  params = opt.best_params_
  mlp = MLPClassifier(hidden_layer_sizes=(256,128,64,32), activation=params['activation'], random_state=params['random_state'], solver=params['solver']).fit(X_trainscaled, y_train)
  y_pred = mlp.predict(X_testscaled)
  print(classification_report(y_test,y_pred))

  calculate_scores(mlp, xgiven, ygiven, xtest)

**Non-binary**

All features

In [12]:
X = df.drop(['tds', 'tds_binary'], axis=1)
y = df.tds

oversample = SMOTE(k_neighbors=2)
X, y = oversample.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [13]:
mlp_class(X_train, y_train, X_test, y_test, X, y)

val. score: 0.9560248780961853
best_params: OrderedDict([('activation', 'tanh'), ('random_state', 3), ('solver', 'adam')]) 

test score: 0.9655263435751241
              precision    recall  f1-score   support

           0       1.00      0.89      0.94        55
           1       0.92      1.00      0.96        60
           2       0.98      1.00      0.99        65

    accuracy                           0.97       180
   macro avg       0.97      0.96      0.96       180
weighted avg       0.97      0.97      0.97       180

f1_macro 0.72056728093402 ± 0.2597692753257206
accuracy 0.9222222222222222 ± 0.08027729719194862
recall 0.7481042450650295 ± 0.24588778650128967


1d

In [14]:
X = df.drop(['tds', 'tds_binary'], axis=1)
y = df.tds

oversample = SMOTE(k_neighbors=2)
X, y = oversample.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [15]:
mlp_class(X_train, y_train, X_test, y_test, X, y)

val. score: 0.9559720179777069
best_params: OrderedDict([('activation', 'tanh'), ('random_state', 2), ('solver', 'adam')]) 

test score: 0.954425539390061
              precision    recall  f1-score   support

           0       1.00      0.96      0.98        55
           1       0.97      1.00      0.98        60
           2       1.00      1.00      1.00        65

    accuracy                           0.99       180
   macro avg       0.99      0.99      0.99       180
weighted avg       0.99      0.99      0.99       180

f1_macro 0.6659708636099125 ± 0.2672861674228746
accuracy 0.9244444444444444 ± 0.059793884653660936
recall 0.692318610946062 ± 0.28997102952351567


**Binary**

All features

In [16]:
X = df.drop(['tds', 'tds_binary'], axis=1)
y = df.tds_binary

oversample = SMOTE(k_neighbors=2)
X, y = oversample.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [17]:
mlp_class(X_train, y_train, X_test, y_test, X, y)

val. score: 0.9554377945922198
best_params: OrderedDict([('activation', 'relu'), ('random_state', 2), ('solver', 'adam')]) 

test score: 0.9317988064791134
              precision    recall  f1-score   support

           0       1.00      0.87      0.93        55
           1       0.90      1.00      0.95        65

    accuracy                           0.94       120
   macro avg       0.95      0.94      0.94       120
weighted avg       0.95      0.94      0.94       120

f1_macro 0.8346475832589618 ± 0.1948400115346957
accuracy 0.95 ± 0.05627314338711376
recall 0.8649035423808563 ± 0.20509336713807536
roc_auc_score 0.8953846153846154


1d

In [18]:
X = df1d.drop(['tds', 'tds_binary'], axis=1)
y = df1d.tds_binary

oversample = SMOTE(k_neighbors=2)
X, y = oversample.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [19]:
mlp_class(X_train, y_train, X_test, y_test, X, y)

val. score: 0.8383789643948715
best_params: OrderedDict([('activation', 'relu'), ('random_state', 1), ('solver', 'adam')]) 

test score: 0.7746086956521739
              precision    recall  f1-score   support

           0       0.98      0.89      0.93        55
           1       0.91      0.98      0.95        65

    accuracy                           0.94       120
   macro avg       0.95      0.94      0.94       120
weighted avg       0.94      0.94      0.94       120

f1_macro 0.724490413165371 ± 0.23795284773150016
accuracy 0.9266666666666665 ± 0.08
recall 0.7542770719903207 ± 0.23983738284548106
roc_auc_score 0.5356643356643356
